In [2]:
# libraries
import numpy as np

In [22]:
# other test variables, which are self parameters
n_total_ftr = 0
target_values = [1,2,3,4,5,6,7,8,9,0]
mu = np.ones((n_total_ftr, len(target_values))) * 0
sigma = np.ones((n_total_ftr, len(target_values))) * 1
penalty_s = 0.01
penalty_r = 0.01
epochs = 1
lr_mu = 0.01
lr_sigma = 0.01
monte_carlo = 10000

# maybe set param amount_classes from target_values

In [ ]:
def __help_softmax()

In [ ]:
def __softmax(x,y): # needs self in model
    """
        Update the distribution parameters mu and sigma by optimizing them in terms of the (log) likelihood.
        Here we assume a Bernoulli distributed target variable. We use a Probit model as our base model.
        This corresponds to the FIRES-GLM model in the paper.

        :param x: (np.ndarray) Batch of observations (numeric values only, consider normalizing data for better results)
        :param y: (np.ndarray) Batch of labels: type integer e.g. 1,2,3,4 usw
     """

     for epoch in range(epochs): #changed to self.epoch in model
         # Shuffle the observations
         random_idx = np.random.permutation(len(y))
         x = x[random_idx]
         y = y[random_idx]
         
         # Iterative update of mu and sigma
         try:
             for k in range(len(target_values)): #loop over all classes



In [8]:
test = np.ones((5,4))
test[:,0]


array([1., 1., 1., 1., 1.])

In [20]:
#for getting gaussnorm r ~ N(0,1)
np.random.randn(5)

array([ 0.08372127, -0.8469425 , -0.18256183,  0.46443678,  0.93265726])

In [ ]:
# get theta at time t and for all featurs for class c is number of feature
theta_c = sigma[:,c] * np.random.randn(total_features) + mu[:,c]